# Segmenting and clustering neighborhoods in city toronto

Downloading all the dependencies that we need

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Download and explore dataset

In [2]:
#import the library used to query a website
import urllib.request #if you are using python3+ version, import urllib.request

In [17]:
wiki ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### Query the website and return the html to the variable 'page'

In [18]:
#Query the website and return the html to the variable 'page'
page = urllib.request.urlopen(wiki) #For python 3 use urllib.request.urlopen(wiki)
print(page)

### Import the Beautiful soup functions to parse the data returned from the website

In [19]:
#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

### Parse the html in the 'page' variable, and store it in Beautiful Soup format

In [20]:
#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page)
print(soup)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","w

### Extract Table

In [21]:
all_tables=soup.find_all('table')
print(all_tables)

[<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td

### Find the right table¶

In [22]:
right_table=soup.find('table', class_='wikitable sortable')
print(right_table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

### Extract the information to DataFrame

In [23]:
#Generate lists
A=[]
B=[]
d=[]
for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        d.append(cells[2].find(text=True))
print(A)
print(B)
print(d)

['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M5A', 'M6A', 'M6A', 'M7A', 'M8A', 'M9A', 'M1B', 'M1B', 'M2B', 'M3B', 'M4B', 'M4B', 'M5B', 'M5B', 'M6B', 'M7B', 'M8B', 'M9B', 'M9B', 'M9B', 'M9B', 'M9B', 'M1C', 'M1C', 'M1C', 'M2C', 'M3C', 'M3C', 'M4C', 'M5C', 'M6C', 'M7C', 'M8C', 'M9C', 'M9C', 'M9C', 'M9C', 'M1E', 'M1E', 'M1E', 'M2E', 'M3E', 'M4E', 'M5E', 'M6E', 'M7E', 'M8E', 'M9E', 'M1G', 'M2G', 'M3G', 'M4G', 'M5G', 'M6G', 'M7G', 'M8G', 'M9G', 'M1H', 'M2H', 'M3H', 'M3H', 'M3H', 'M4H', 'M5H', 'M5H', 'M5H', 'M6H', 'M6H', 'M7H', 'M8H', 'M9H', 'M1J', 'M2J', 'M2J', 'M2J', 'M3J', 'M3J', 'M4J', 'M5J', 'M5J', 'M5J', 'M6J', 'M6J', 'M7J', 'M8J', 'M9J', 'M1K', 'M1K', 'M1K', 'M2K', 'M3K', 'M3K', 'M4K', 'M4K', 'M5K', 'M5K', 'M6K', 'M6K', 'M6K', 'M7K', 'M8K', 'M9K', 'M1L', 'M1L', 'M1L', 'M2L', 'M2L', 'M3L', 'M4L', 'M4L', 'M5L', 'M5L', 'M6L', 'M6L', 'M6L', 'M7L', 'M8L', 'M9L', 'M1M', 'M1M', 'M1M', 'M2M', 'M2M', 'M3M', 'M4M', 'M5M', 'M5M', 'M6M', 'M6M', 'M6M', 'M6M', 'M7M', 'M8M', 'M9M', 'M9M', 'M1N', 'M1N', 'M2N'

### Data wrangling

In [25]:
for n, i in enumerate(B):
    if i == 'Not assigned':
        B[n] = 'NaN'
print(B)

['NaN', 'NaN', 'North York', 'North York', 'Downtown Toronto', 'Downtown Toronto', 'North York', 'North York', "Queen's Park", 'NaN', 'Etobicoke', 'Scarborough', 'Scarborough', 'NaN', 'North York', 'East York', 'East York', 'Downtown Toronto', 'Downtown Toronto', 'North York', 'NaN', 'NaN', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Scarborough', 'Scarborough', 'Scarborough', 'NaN', 'North York', 'North York', 'East York', 'Downtown Toronto', 'York', 'NaN', 'NaN', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Scarborough', 'Scarborough', 'Scarborough', 'NaN', 'NaN', 'East Toronto', 'Downtown Toronto', 'York', 'NaN', 'NaN', 'NaN', 'Scarborough', 'NaN', 'NaN', 'East York', 'Downtown Toronto', 'Downtown Toronto', 'NaN', 'NaN', 'NaN', 'Scarborough', 'North York', 'North York', 'North York', 'North York', 'East York', 'Downtown Toronto', 'Downtown Toronto', 'Downtown Toronto', 'West Toronto', 'West Toronto', 'NaN', 'NaN', 'NaN', 'Scarborough', 'North York', 'No

In [26]:
for n, i in enumerate(d):
    if i == 'Not assigned\n':
        d[n] = 'NaN'
print(d)

['NaN', 'NaN', 'Parkwoods', 'Victoria Village', 'Harbourfront', 'Regent Park', 'Lawrence Heights', 'Lawrence Manor', 'NaN', 'NaN', 'Islington Avenue', 'Rouge', 'Malvern', 'NaN', 'Don Mills North\n', 'Woodbine Gardens', 'Parkview Hill', 'Ryerson\n', 'Garden District\n', 'Glencairn\n', 'NaN', 'NaN', 'Cloverdale\n', 'Islington', 'Martin Grove\n', 'Princess Gardens', 'West Deane Park', 'Highland Creek', 'Rouge Hill', 'Port Union', 'NaN', 'Flemingdon Park', 'Don Mills South\n', 'Woodbine Heights', 'St. James Town', 'Humewood-Cedarvale', 'NaN', 'NaN', 'Bloordale Gardens\n', 'Eringate\n', 'Markland Wood', 'Old Burnhamthorpe\n', 'Guildwood\n', 'Morningside', 'West Hill', 'NaN', 'NaN', 'The Beaches', 'Berczy Park', 'Caledonia-Fairbanks\n', 'NaN', 'NaN', 'NaN', 'Woburn', 'NaN', 'NaN', 'Leaside', 'Central Bay Street\n', 'Christie\n', 'NaN', 'NaN', 'NaN', 'Cedarbrae\n', 'Hillcrest Village', 'Bathurst Manor', 'Downsview North\n', 'Wilson Heights', 'Thorncliffe Park', 'Adelaide\n', 'King\n', 'Richmo

In [27]:
C = []
for i in d:
    C.append(i.strip())
C

['NaN',
 'NaN',
 'Parkwoods',
 'Victoria Village',
 'Harbourfront',
 'Regent Park',
 'Lawrence Heights',
 'Lawrence Manor',
 'NaN',
 'NaN',
 'Islington Avenue',
 'Rouge',
 'Malvern',
 'NaN',
 'Don Mills North',
 'Woodbine Gardens',
 'Parkview Hill',
 'Ryerson',
 'Garden District',
 'Glencairn',
 'NaN',
 'NaN',
 'Cloverdale',
 'Islington',
 'Martin Grove',
 'Princess Gardens',
 'West Deane Park',
 'Highland Creek',
 'Rouge Hill',
 'Port Union',
 'NaN',
 'Flemingdon Park',
 'Don Mills South',
 'Woodbine Heights',
 'St. James Town',
 'Humewood-Cedarvale',
 'NaN',
 'NaN',
 'Bloordale Gardens',
 'Eringate',
 'Markland Wood',
 'Old Burnhamthorpe',
 'Guildwood',
 'Morningside',
 'West Hill',
 'NaN',
 'NaN',
 'The Beaches',
 'Berczy Park',
 'Caledonia-Fairbanks',
 'NaN',
 'NaN',
 'NaN',
 'Woburn',
 'NaN',
 'NaN',
 'Leaside',
 'Central Bay Street',
 'Christie',
 'NaN',
 'NaN',
 'NaN',
 'Cedarbrae',
 'Hillcrest Village',
 'Bathurst Manor',
 'Downsview North',
 'Wilson Heights',
 'Thorncliffe Par

In [28]:
l = {'Postcode': A, 'Borough': B, 'Neighbourhood': d}
df = pd.DataFrame(data=l)
df

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN
9,M8A,NaN,NaN


### Ignore cells with a borough that is Not assigned

In [29]:
df = df[df.Borough != 'NaN']
df.head(7)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN


In [30]:
# reset index
df.reset_index(drop=True, inplace=True)
df.head(8)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,NaN
7,M9A,Etobicoke,Islington Avenue


### Replace the neighbourhood with not assigned same as borough

In [31]:
import numpy as np
df['Neighbourhood'].replace('NaN', df['Borough'],inplace=True)
df.head(8)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue


### Group neighbourhood for same postcode area

In [32]:
df = df.groupby('Postcode').agg({'Borough':'first','Neighbourhood': ', '.join}).reset_index()
df = df.replace('\n','', regex=True)
df.head(8)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"


In [33]:
df.shape

(103, 3)

### Adding two more columns to our dataframe i.e. latitude and longitude

In [34]:
df['Latitude']= ""
df['Longitude']= ""
df.head(9)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",,
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",,
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,
5,M1J,Scarborough,Scarborough Village,,
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",,
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",,
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",,


In [36]:
!conda install -c conda-forge geopy --yes

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [37]:
CLIENT_ID = 'UXZ5RKNVTDG3TH5WK3S24IEU3UJ21V4VU1ZCV1HEV0LDCDA1'# your Foursquare ID
CLIENT_SECRET = 'AAX3NKUYG1TPARXWG03FQ00TTMBHOT1FX30ROV2VPMX40LGU' # your Foursquare id
VERSION = '20180604'
LIMIT = 30

### Calling geolocator to find latitude and longitude

In [38]:
for index, row in df.iterrows():
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(row['Borough'])
    row['Latitude'] = location.latitude
    row['Longitude'] = location.longitude

### csv file to create the dataframe

In [39]:
df.to_csv('toronto.csv')

In [40]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",54.2848,-0.409034
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",54.2848,-0.409034
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",54.2848,-0.409034
3,M1G,Scarborough,Woburn,54.2848,-0.409034
4,M1H,Scarborough,Cedarbrae,54.2848,-0.409034
5,M1J,Scarborough,Scarborough Village,54.2848,-0.409034
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",54.2848,-0.409034
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",54.2848,-0.409034
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",54.2848,-0.409034
9,M1N,Scarborough,"Birch Cliff, Cliffside West",54.2848,-0.409034


In [41]:
import folium

### Analyzing neighbourhoods of toronto

In [42]:
print('The dataframe has {} Boroughs and {} Neighbourhood.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 Boroughs and 103 Neighbourhood.


### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below

In [43]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top.

In [44]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Let's define a function to get nearby venues of a borough

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [46]:
Toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

### Let's check the size of the resulting dataframe

In [47]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2277, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",54.28476,-0.409034,Gianni's,54.283031,-0.405256,Italian Restaurant
1,"Rouge, Malvern",54.28476,-0.409034,Eat Me Cafe,54.280697,-0.406778,Restaurant
2,"Rouge, Malvern",54.28476,-0.409034,Aldi,54.281965,-0.407199,Supermarket
3,"Rouge, Malvern",54.28476,-0.409034,Domino's Pizza,54.283844,-0.403644,Pizza Place
4,"Rouge, Malvern",54.28476,-0.409034,Pizza Hut,54.280872,-0.407803,Pizza Place


### Let's check how many venues were returned for each neighborhood

In [48]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Agincourt,6,6,6,6,6,6
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",6,6,6,6,6,6
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",5,5,5,5,5,5
"Alderwood, Long Branch",5,5,5,5,5,5
"Bathurst Manor, Downsview North, Wilson Heights",30,30,30,30,30,30
Bayview Village,30,30,30,30,30,30
"Bedford Park, Lawrence Manor East",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30


### Let's find out how many unique categories can be curated from all the returned venues

In [49]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 90 uniques categories.


### Analyze Each Neighborhood

In [50]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Bakery,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Farmers Market,French Restaurant,Furniture / Home Store,Garden,Gastropub,Greek Restaurant,Grocery Store,Historic Site,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lingerie Store,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Music Venue,Neighborhood,Office,Opera House,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Theater,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### And let's examine the new dataframe size

In [51]:
Toronto_onehot.shape

(2277, 91)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [52]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Bakery,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Farmers Market,French Restaurant,Furniture / Home Store,Garden,Gastropub,Greek Restaurant,Grocery Store,Historic Site,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lingerie Store,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Music Venue,Neighborhood,Office,Opera House,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Theater,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Adelaide, King, Richmond",0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.066667,0.000000,0.033333,0.000000,0.000000,0.000000
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0

### Let's confirm the new size

In [54]:
Toronto_grouped.shape

(103, 91)

### Let's print each neighborhood along with the top 5 most common venues

In [55]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0       Clothing Store  0.10
1  American Restaurant  0.07
2              Theater  0.07
3                Plaza  0.07
4       Cosmetics Shop  0.07


----Agincourt----
                venue  freq
0         Pizza Place  0.33
1          Restaurant  0.17
2  Italian Restaurant  0.17
3       Grocery Store  0.17
4         Supermarket  0.17


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0         Pizza Place  0.33
1          Restaurant  0.17
2  Italian Restaurant  0.17
3       Grocery Store  0.17
4         Supermarket  0.17


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0           Playground   0.4
1               Garden   0.2
2          Coffee Shop   0.2
3          Supermarket   0.2
4  American Restaurant   0.0


----Alderwood, Long Branch----
                 venue  freq
0           Pl

### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [57]:
import numpy as np
import pandas as pd
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
1,Agincourt,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
4,"Alderwood, Long Branch",Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner


### Cluster Neighborhoods

In [58]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 1, 1, 3, 3, 3, 0, 2], dtype=int32)

## Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
3,M1G,Scarborough,Woburn,54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
4,M1H,Scarborough,Cedarbrae,54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


### Finally, let's visualize the resulting clusters

In [60]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

### Cluster 1

In [61]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
51,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
52,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
53,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
54,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
55,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
56,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
57,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
58,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue
59,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Plaza,Cosmetics Shop,Burrito Place,Shopping Mall,Department Store,Comic Shop,Music Venue


In [62]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
89,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
90,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
91,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
92,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
93,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
94,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
95,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
99,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner
100,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Yoga Studio,Farmers Market,Deli / Bodega,Department Store,Dessert Shop,Diner


In [63]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
1,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
2,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
3,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
4,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
5,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
6,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
7,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
8,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
9,Scarborough,2,Pizza Place,Italian Restaurant,Restaurant,Grocery Store,Supermarket,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [64]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool
18,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool
19,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool
20,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool
21,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool
22,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool
23,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool
24,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool
25,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool
26,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Plaza,Burrito Place,Pool


In [65]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
41,East Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
42,East Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
43,East Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
44,Central Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
45,Central Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
46,Central Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
47,Central Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
48,Central Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
49,Central Toronto,4,Japanese Restaurant,Chinese Restaurant,Jazz Club,Concert Hall,Plaza,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant
